In [21]:
using ExcelReaders

fPF = openxl("PF.xlsx")
fPF1 = openxl("PF1.xlsx")

include("case14test.m")

Samples=size(readxlsheet(fPF, "Sheet1"),1)/size(bus,1)

ResRealImag=zeros(Samples,2size(bus,1))
ResMagPha=zeros(Samples,2size(bus,1))

for it in Int(Samples)-144:Int(Samples)
    PF = readxl(fPF, "Sheet1!A$(14*it-13):G$(14*it)")
    PF1 = readxl(fPF1, "Sheet1!A$(20*it-19):G$(20*it)")
    E=PF[:,2].*cos((π/180)PF[:,3])+PF[:,2].*sin((π/180)PF[:,3])im
Iₗ=complex(zeros(size(branch,1),2))
for i in 1:size(branch,1)
    Iₗ[i,1]=((PF1[i,4]/baseMVA)+(PF1[i,5]/baseMVA)im)/E[Int(PF1[i,2])]
    Iₗ[i,2]=((PF1[i,6]/baseMVA)+(PF1[i,7]/baseMVA)im)/E[Int(PF1[i,3])]
end

PMU=[2, 6, 7, 9]
r=branch[:,3];x=branch[:,4];y=1./(r+x*im);b=branch[:,5];y₀=b/2*im;
nLinePMU=0
for k in 1:size(branch,1)
    for k′ in 1:size(PMU,1)
        if branch[k,1]==PMU[k′]||branch[k,2]==PMU[k′]
            for k′′ in 1:size(PMU,1)
                if branch[k,1]==PMU[k′]&&branch[k,2]==PMU[k′′]
                    nLinePMU=nLinePMU+1
                end
            end
        end
    end
end
A=zeros(size(branch,1)+nLinePMU,size(bus,1))
Y=complex(zeros(size(branch,1)+nLinePMU,size(branch,1)+nLinePMU));
yₛ=complex(zeros(size(branch,1)+nLinePMU,size(bus,1)));y₀=[y₀;zeros(nLinePMU,1)]
I₎=complex(zeros(size(branch,1)+nLinePMU,1));
a′=0
for k in 1:size(branch,1)
    for k′ in 1:size(PMU,1)
        if branch[k,1]==PMU[k′]||branch[k,2]==PMU[k′]
            if branch[k,1]==PMU[k′]&&branch[k,2]!=PMU[k′]
                A[k+a′,PMU[k′]]=1
                A[k+a′,Int(branch[k,2])]=-1
                Y[k+a′,k+a′]=y[k]
                yₛ[k+a′,PMU[k′]]=y₀[k]
                I₎[k+a′]=Iₗ[k,1]
            elseif branch[k,2]==PMU[k′]&&branch[k,1]!=PMU[k′]
                A[k+a′,PMU[k′]]=1
                A[k+a′,Int(branch[k,1])]=-1
                Y[k+a′,k+a′]=y[k]
                yₛ[k+a′,PMU[k′]]=y₀[k]
                I₎[k+a′]=Iₗ[k,2]
            end
            for k′′ in 1:size(PMU,1)
                if branch[k,1]==PMU[k′]&&branch[k,2]==PMU[k′′]
                    A[k+a′,PMU[k′]]=1
                    A[k+a′,Int(branch[k,2])]=-1
                    Y[k+a′,k+a′]=y[k]
                    yₛ[k+a′,PMU[k′]]=y₀[k]
                    I₎[k+a′]=Iₗ[k,1]
                    A[k+a′+1,PMU[k′′]]=1
                    A[k+a′+1,Int(branch[k,1])]=-1
                    Y[k+a′+1,k+a′+1]=y[k]
                    yₛ[k+a′+1,PMU[k′′]]=y₀[k]
                    I₎[k+a′+1]=Iₗ[k,2]
                    a′=a′+1
                end
            end
        end
    end
end
A′=A
A=A[find(A[i,:]!=zeros(size(A,2)) for i in 1:size(A,1)),:]
Y=Y[find(Y[i,:]!=zeros(size(Y,2)) for i in 1:size(Y,1)),find(Y[i,:]!=zeros(size(Y,2)) for i in 1:size(Y,1))]
yₛ=yₛ[find(A′[i,:]!=zeros(size(A′,2)) for i in 1:size(A′,1)),find(A′[:,i]!=zeros(size(A′,1)) for i in 1:size(A′,2))]
ΙΙ=zeros(size(PMU,1),size(bus,1))
a′′=0
for i in 1:size(bus,1)
    for j in 1:size(PMU,1)
        if i==PMU[j]
            ΙΙ[1+a′′,i]=1
            a′′=a′′+1
        end
    end
end
B=[ΙΙ;Y*A+yₛ]
W=1000*eye(size(B,1))
z=B*E+(2/3000)*B*E.*(rand(size(B,1))+rand(size(B,1))im)
M=(transpose(B)*(W^(-1))*B)^(-1)*transpose(B)*(W^(-1))
#c=[0 -.1 zeros(1,12)]'.*E#2
#c=[zeros(1,5) -.1 zeros(1,8)]'.*E#6
#c=[zeros(1,6) .1 zeros(1,7)]'.*E#7
c=[zeros(1,8) -.1 zeros(1,5)]'.*E#9
z=z+B*c
x̂=M*z
ResRealImag[it,:]=[real(x̂)' imag(x̂)']
ResMagPha[it,:]=[abs(x̂)' angle(x̂)']
end

writecsv( "ResRealImagAttack9Inc.csv",  ResRealImag)
writecsv( "ResMagPhaAttack9Inc.csv",  ResMagPha)

# 118 bus

In [2]:
include("PowerFlowRes118.jl")
include("case118.m")
Samples=2045
ResRealImag=zeros(144,2size(bus,1))#Samples->144
ResMagPha=zeros(144,2size(bus,1))
PMU=[2, 5, 9, 12, 15, 17, 21, 25, 29, 34, 37, 42, 45, 49, 53, 56, 62, 63, 68, 70, 71, 75, 77, 80, 85, 86, 91, 94, 102, 105, 110, 114]
# Xu J, Wen MH, Li VO, Leung K-C. Optimal PMU placement for wide-area
#monitoring using chemical reaction optimization. In: Innovative smart grid
#technologies 2013 (ISGT), IEEE PES: IEEE; 2013. p. 1–6.
c=zeros(size(bus,1),1)
for ItPMU in 1:size(PMU,1)
    E=PF[:,2].*cos((π/180)PF[:,3])+PF[:,2].*sin((π/180)PF[:,3])im
c[PMU[ItPMU]]=0.1
    c=c.*E
Iₗ=complex(zeros(size(branch,1),2))
for i in 1:size(branch,1)
    Iₗ[i,1]=((PF1[i,4]/baseMVA)+(PF1[i,5]/baseMVA)im)/E[Int(PF1[i,2])]
    Iₗ[i,2]=((PF1[i,6]/baseMVA)+(PF1[i,7]/baseMVA)im)/E[Int(PF1[i,3])]
end
#PMU=[3, 9, 11, 12, 17, 21, 25, 28, 34, 37, 40, 45, 49, 53, 56, 62, 72, 75, 77, 80, 85, 86, 90, 94, 102, 105, 110, 114]
r=branch[:,3];x=branch[:,4];y=1./(r+x*im);b=branch[:,5];y₀=b/2*im;
nLinePMU=0
for k in 1:size(branch,1)
    for k′ in 1:size(PMU,1)
        if branch[k,1]==PMU[k′]||branch[k,2]==PMU[k′]
            for k′′ in 1:size(PMU,1)
                if branch[k,1]==PMU[k′]&&branch[k,2]==PMU[k′′]
                    nLinePMU=nLinePMU+1
                end
            end
        end
    end
end
A=zeros(size(branch,1)+nLinePMU,size(bus,1))
Y=complex(zeros(size(branch,1)+nLinePMU,size(branch,1)+nLinePMU));
yₛ=complex(zeros(size(branch,1)+nLinePMU,size(bus,1)));y₀=[y₀;zeros(nLinePMU,1)]
I₎=complex(zeros(size(branch,1)+nLinePMU,1));
a′=0
for k in 1:size(branch,1)
    for k′ in 1:size(PMU,1)
        if branch[k,1]==PMU[k′]||branch[k,2]==PMU[k′]
            if branch[k,1]==PMU[k′]&&branch[k,2]!=PMU[k′]
                A[k+a′,PMU[k′]]=1
                A[k+a′,Int(branch[k,2])]=-1
                Y[k+a′,k+a′]=y[k]
                yₛ[k+a′,PMU[k′]]=y₀[k]
                I₎[k+a′]=Iₗ[k,1]
            elseif branch[k,2]==PMU[k′]&&branch[k,1]!=PMU[k′]
                A[k+a′,PMU[k′]]=1
                A[k+a′,Int(branch[k,1])]=-1
                Y[k+a′,k+a′]=y[k]
                yₛ[k+a′,PMU[k′]]=y₀[k]
                I₎[k+a′]=Iₗ[k,2]
            end
            for k′′ in 1:size(PMU,1)
                if branch[k,1]==PMU[k′]&&branch[k,2]==PMU[k′′]
                    A[k+a′,PMU[k′]]=1
                    A[k+a′,Int(branch[k,2])]=-1
                    Y[k+a′,k+a′]=y[k]
                    yₛ[k+a′,PMU[k′]]=y₀[k]
                    I₎[k+a′]=Iₗ[k,1]
                    A[k+a′+1,PMU[k′′]]=1
                    A[k+a′+1,Int(branch[k,1])]=-1
                    Y[k+a′+1,k+a′+1]=y[k]
                    yₛ[k+a′+1,PMU[k′′]]=y₀[k]
                    I₎[k+a′+1]=Iₗ[k,2]
                    a′=a′+1
                end
            end
        end
    end
end
A′=A
A=A[find(A[i,:]!=zeros(size(A,2)) for i in 1:size(A,1)),:]
Y=Y[find(Y[i,:]!=zeros(size(Y,2)) for i in 1:size(Y,1)),find(Y[i,:]!=zeros(size(Y,2)) for i in 1:size(Y,1))]
yₛ=yₛ[find(A′[i,:]!=zeros(size(A′,2)) for i in 1:size(A′,1)),find(A′[:,i]!=zeros(size(A′,1)) for i in 1:size(A′,2))]
ΙΙ=zeros(size(PMU,1),size(bus,1))
a′′=0
for i in 1:size(bus,1)
    for j in 1:size(PMU,1)
        if i==PMU[j]
            ΙΙ[1+a′′,i]=1
            a′′=a′′+1
        end
    end
end
B=[ΙΙ;Y*A+yₛ]
W=1000*eye(size(B,1))
for it in 1:144#Samples-144:Samples
z=B*E+(2/3000)*B*E.*(rand(size(B,1))+rand(size(B,1))im)
M=(transpose(B)*(W^(-1))*B)^(-1)*transpose(B)*(W^(-1))
#c=[0 .1 zeros(1,12)]'.*E#2
#c=[0 0 0 0 zeros(size(branch,1))]#6
#c=[0 0 0 0 zeros(size(branch,1))]#7
#c=[0 0 0 0 zeros(size(branch,1))]#9
z=z+B*c
x̂=M*z
#[abs(x̂) angle(x̂)]

################
#sqrt(sum((angle(x̂)-angle(E)).^2)/size(B,2)),sqrt(sum((abs(x̂)-abs(E)).^2)/size(B,2))
#sqrt(sum((x̂-E).^2)/size(B,2))
#sqrt(([sum([angle(x̂)-angle(E)]);sum([abs(x̂)-abs(E)])].^2)/size(B,2))
#I₎=I₎[find(I₎)]
#zₜ=[E[PMU];I₎]
ResRealImag[it,:]=[real(x̂)' imag(x̂)']
ResMagPha[it,:]=[abs(x̂)' angle(x̂)']
end

writecsv( "ResRealImagAttack$(PMU[ItPMU])Dec.csv",  ResRealImag)
writecsv( "ResMagPhaAttack$(PMU[ItPMU])Dec.csv",  ResMagPha)
end